<a href="https://www.kaggle.com/code/drkaggle22/toxic-comments-classification-tf-idf-nltk?scriptVersionId=176830496" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/toxic-comments-transformer-embeddings/test_embs_large_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_large_320.npy
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_large_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_320.npy
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_320.npy
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_large_320.npy
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


In [2]:
df_train = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip', index_col='id')

df_test = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip', index_col='id')

In [3]:
pd.set_option('display.max_rows', None)

In [4]:
df_train['comment_text'] = df_train['comment_text'].str.lower()
df_test['comment_text'] = df_test['comment_text'].str.lower()

In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 153164 entries, 00001cee341fdb12 to ffffce3fb183ee80
Data columns (total 1 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   comment_text  153164 non-null  object
dtypes: object(1)
memory usage: 2.3+ MB


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159571 entries, 0000997932d777bf to fff46fc426af1f9a
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   comment_text   159571 non-null  object
 1   toxic          159571 non-null  int64 
 2   severe_toxic   159571 non-null  int64 
 3   obscene        159571 non-null  int64 
 4   threat         159571 non-null  int64 
 5   insult         159571 non-null  int64 
 6   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 9.7+ MB


In [7]:
import re

def remove_special_characters(text):
    text = re.sub(r'http\S+', ' ', text )
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\bhttps?://[a-zA-Z0-9-]+(?:\.[a-zA-Z0-9-]+)+\b', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\d', ' ', text)  # Corrected line
    text= re.sub(r'[\u4e00-\u9fff]+', ' ', text)
    return text

df_train['comment_text'] = df_train['comment_text'].apply(remove_special_characters)
df_test['comment_text'] = df_test['comment_text'].apply(remove_special_characters)

In [8]:
file_path = '/kaggle/input/toxic-comments-transformer-embeddings/train_embs_320.npy'
df_train_emb = np.load(file_path)

In [9]:
(df_train_emb).shape

(159571, 768)

In [10]:
X = df_train_emb  # Assuming 'comment_text' contains your text data
y = df_train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

In [11]:
from sklearn.model_selection import train_test_split
# Splitting the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=42)

In [12]:
x = X_train
val_x = X_valid

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer


# Initialize MultiLabelBinarizer with the classes
mlb = MultiLabelBinarizer()

# Transform target labels into binary form
y_train_binary = mlb.fit_transform(y_train)
y_valid_binary = mlb.transform(y_valid)

# At this point, y_train_binary and y_valid_binary are ready to be used in your model training


In [14]:
import numpy as np

# List of class labels
class_labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Initialize empty arrays to store binary labels
y_train_binary = np.zeros((len(y_train), len(class_labels)), dtype=int)
y_valid_binary = np.zeros((len(y_valid), len(class_labels)), dtype=int)

# Manually create binary labels for each class
for i, label in enumerate(class_labels):
    y_train_binary[:, i] = y_train[label]
    y_valid_binary[:, i] = y_valid[label]

# At this point, y_train_binary and y_valid_binary are ready to be used in your model training


In [15]:
print("y_train_binary shape:", y_train_binary.shape)
print("y_valid_binary shape:", y_valid_binary.shape)


y_train_binary shape: (127656, 6)
y_valid_binary shape: (31915, 6)


In [16]:
from sklearn.neural_network import MLPClassifier
import numpy as np

epochs = 50  # Define the number of epochs
# Set up the MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(200, 100, 64, 6),
                    activation='relu',
                    solver='adam',
                    random_state=42,
                    verbose=True,
                    learning_rate_init=0.001,
                    max_iter=epochs,  
                    early_stopping=True,  # Enable early stopping
                    n_iter_no_change=5,    # Number of iterations with no improvement to wait before stopping
                    )

# Fit the classifier with the training data and binary target labels
clf.fit(X_train, y_train_binary)


Iteration 1, loss = 1.59501357
Validation score: 0.908037
Iteration 2, loss = 0.37100801
Validation score: 0.910074
Iteration 3, loss = 0.35017820
Validation score: 0.911484
Iteration 4, loss = 0.33606160
Validation score: 0.912032
Iteration 5, loss = 0.33006005
Validation score: 0.913520
Iteration 6, loss = 0.32666108
Validation score: 0.913912
Iteration 7, loss = 0.31875338
Validation score: 0.910935
Iteration 8, loss = 0.31464555
Validation score: 0.912032
Iteration 9, loss = 0.31285178
Validation score: 0.915165
Iteration 10, loss = 0.30738480
Validation score: 0.911954
Iteration 11, loss = 0.30549278
Validation score: 0.912424
Iteration 12, loss = 0.30338341
Validation score: 0.913364
Iteration 13, loss = 0.30046855
Validation score: 0.912502
Iteration 14, loss = 0.29775815
Validation score: 0.912189
Iteration 15, loss = 0.29437951
Validation score: 0.913520
Validation score did not improve more than tol=0.000100 for 5 consecutive epochs. Stopping.


MLPClassifier(early_stopping=True, hidden_layer_sizes=(200, 100, 64, 6),
              max_iter=50, n_iter_no_change=5, random_state=42, verbose=True)

In [17]:
file_path_2 = '/kaggle/input/toxic-comments-transformer-embeddings/test_embs_320.npy'

df_test_emb = np.load(file_path_2)

In [18]:
df_test_emb.shape

(153164, 768)

In [19]:
# Assuming clf is the trained MLPClassifier model

# Predict probabilities for test data
y_test_prob = clf.predict_proba(df_test_emb)

# Print the shape of predicted probabilities for confirmation
print("Shape of predicted probabilities:", y_test_prob.shape)

# Now you have predicted probabilities for each class for the test data


Shape of predicted probabilities: (153164, 6)


In [20]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [21]:
# Convert the predicted probabilities to float
y_test_prob_float = y_test_prob.astype(float)

# Print the updated type of predicted probabilities for confirmation
print("Type of predicted probabilities:", y_test_prob_float.dtype)

# Now y_test_prob_float contains predicted probabilities for each class for each data point in the test data, stored as float numbers


Type of predicted probabilities: float64


In [22]:
import numpy as np

# Assuming y_test_prob_float is your array of predicted probabilities

# Set numpy printing options to suppress scientific notation
np.set_printoptions(suppress=True)

# Print the updated predicted probabilities
print(y_test_prob_float)


[[0.98605901 0.29890001 0.91988772 0.06856659 0.84363818 0.15350462]
 [0.00021741 0.0000002  0.00008218 0.00000106 0.00001002 0.00000076]
 [0.00144863 0.00000578 0.0008197  0.00001828 0.00011916 0.00001479]
 ...
 [0.00057289 0.00000145 0.00025172 0.00000848 0.00005191 0.00000736]
 [0.00115022 0.00000444 0.0005115  0.00002629 0.00014131 0.00002563]
 [0.92352152 0.02948794 0.62648517 0.00787189 0.39272907 0.01693846]]


In [23]:
# Set the floating-point format for display
pd.options.display.float_format = '{:.10f}'.format

# Create submission DataFrame
submid = pd.DataFrame({'id': df_test.index})  # Use index as 'id' column
submission = pd.concat([submid, pd.DataFrame(y_test_prob_float, columns=classes)], axis=1)

# Write submission DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)


In [24]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.9860590100,0.2989000082,0.9198877215,0.0685665905,0.8436381817,0.1535046250
1,0000247867823ef7,0.0002174065,0.0000002005,0.0000821843,0.0000010607,0.0000100213,0.0000007647
2,00013b17ad220c46,0.0014486257,0.0000057787,0.0008197028,0.0000182788,0.0001191619,0.0000147920
3,00017563c3f7919a,0.0000742317,0.0000000367,0.0000198328,0.0000003750,0.0000036366,0.0000003011
4,00017695ad8997eb,0.0005215177,0.0000003426,0.0000961779,0.0000030745,0.0000385326,0.0000036960
